In [1]:
# Load Libraries
from pandasql import sqldf
import pandas as pd
import numpy as np
import re
import datetime
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import xlwings as xw
import openpyxl
from openpyxl import workbook, load_workbook
from sqlalchemy import null
from zmq import NULL
mysql = lambda q: sqldf(q,globals()) 

import warnings
warnings.filterwarnings("ignore")

In [2]:
start_time = datetime.now()

In [3]:
#Report Date
current_mnth_till = '31-03-2023'

In [4]:
#Report details
report_file_path = r"C:\Users\ank2\Desktop\DAILY MIS REPORT\REPORTS\Automation Test\Booking MIS 31st Mar 2023 - -Automation Test.xlsx"

### Current Month Data - 

In [5]:
#current month Booking data
Booking1 = pd.read_csv(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\BOOKING PROCESSED DUMP\Booking_processed-MTD 31-MARCH 2023 Month End-1.csv", encoding='latin-1',low_memory=False) #BookingDump Data
Booking2 = pd.read_csv(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\BOOKING PROCESSED DUMP\Booking_processed-MTD 31-MARCH 2023 Month End-2.csv", encoding='latin-1',low_memory=False ) #BookingDump Data
Booking = pd.concat([Booking1,Booking2])

In [6]:
#Current month Cancellation Data
Cancellation = pd.read_csv(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\CANCELLATION PROCESSED DUMP\old\cancellation_PROCESSED-MTD 31 MARCH 2023 Month End.csv",encoding='latin-1',low_memory=False)

### Previous Month Data - 

In [7]:
#prev month data
df_a = pd.read_csv(r'C:\Users\ank2\Desktop\DAILY MIS REPORT\PREVIOUS MONTH\old\PrevMonthFebruary-1.csv',encoding='latin-1',low_memory=False)
df_b = pd.read_csv(r'C:\Users\ank2\Desktop\DAILY MIS REPORT\PREVIOUS MONTH\old\PrevMonthFebruary-2.csv',encoding='latin-1',low_memory=False)
df=pd.concat([df_a,df_b])

In [8]:
#Filtering out the data
df_prev_mnth = df[(df['Charterornoncharter'] == 'Non-Charter') & (df['BookDate'] <= current_mnth_till)][['BookDate', 'Dom_Int', 'Region', 'Global_Region', 'City', 'NEWMainChannelforCommercialTeam', 'Sector', 'Country', 'FIT_Grp', 'PAX', 'GrossRev']]

In [9]:
df_prev_mnth['BookDate'].unique()

array(['01-02-2023', '02-02-2023', '03-02-2023', '04-02-2023',
       '05-02-2023', '06-02-2023', '07-02-2023', '08-02-2023',
       '09-02-2023', '10-02-2023', '11-02-2023', '12-02-2023',
       '13-02-2023', '14-02-2023', '15-02-2023', '16-02-2023',
       '17-02-2023', '18-02-2023', '19-02-2023', '20-02-2023',
       '21-02-2023', '22-02-2023', '23-02-2023', '24-02-2023',
       '25-02-2023', '26-02-2023', '27-02-2023', '28-02-2023'],
      dtype=object)

### Previous Year Data - 

In [10]:
#prev year data
df1 = pd.read_csv(r'C:\Users\ank2\Desktop\DAILY MIS REPORT\PREVIOUS YEAR\old\PrevYearMarch.csv',encoding='latin-1',low_memory=False)

In [11]:
#Filtering out data
df_prev_year = df1[(df1['Charterornoncharter'] == 'Non - Charter') & (df1['BookDate'] <= current_mnth_till)][['BookDate', 'DOM/INT', 'Global_Region_GSA', 'Global_Region', 'City', 'NEWMainChannelforCommercialTeam', 'Sector', 'Country', 'FIT_Grp', 'PAX', 'GrossRev']]

In [12]:
#Checking the booking dates
df_prev_year['BookDate'].unique()

array(['01-03-2019', '02-03-2019', '03-03-2019', '04-03-2019',
       '05-03-2019', '06-03-2019', '07-03-2019', '08-03-2019',
       '09-03-2019', '10-03-2019', '11-03-2019', '12-03-2019',
       '13-03-2019', '14-03-2019', '15-03-2019', '16-03-2019',
       '17-03-2019', '18-03-2019', '19-03-2019', '20-03-2019',
       '21-03-2019', '22-03-2019', '23-03-2019', '24-03-2019',
       '25-03-2019', '26-03-2019', '27-03-2019', '28-03-2019',
       '29-03-2019', '30-03-2019', '31-03-2019'], dtype=object)

### Masters - 

In [13]:
#Region City Region_x master
CHMast=pd.read_excel(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\MASTERS\ChannelAndRegionNameMaster.xlsx", sheet_name= 'Channel' )
RNMast=pd.read_excel(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\MASTERS\ChannelAndRegionNameMaster.xlsx", sheet_name= 'Region' )
RegionCityChannelcombo = pd.read_excel(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\MASTERS\AAA-MasterRegionCityChannel.xlsx")
RegionCityChannelkey = pd.read_excel(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\MASTERS\RegionCityChanMaster.xlsx")

#Sector Master
SectorMast = pd.read_excel(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\MASTERS\SectorMaster.xlsx")

#B2B & B2C Masters
B2BMaster=pd.read_excel(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\MASTERS\ChannelAndRegionNameMaster.xlsx", sheet_name= 'B2B' )
B2CMaster=pd.read_excel(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\MASTERS\ChannelAndRegionNameMaster.xlsx", sheet_name= 'B2C' )

In [14]:
#Filtering the cur month booking data
Booking_final = Booking[Booking['charter_noncharter'] == 'Non-Charter']

In [15]:
Booking_final['Gross_Rev'] = Booking_final['Gross_Rev'].apply( lambda x : re.sub(",","", str(x) ) )
Booking_final['Gross_Rev'] = Booking_final['Gross_Rev'].apply( lambda x : float(x)  )
Booking_final['Gross_Rev_inCrs'] = Booking_final['Gross_Rev'].apply( lambda x : x / 10**7 )

In [16]:
#filtering the cur month cancellation data
Cancellation_final = Cancellation[Cancellation['Charter_Noncharter'] == 'Non-Charter']

In [17]:
#Creating a new column 'Week' for weekly average calculation
Booking_final['BkgDate'] = pd.to_datetime( Booking_final['BkgDate'] , format = '%d-%m-%Y')

Booking_final['Week'] = Booking_final['BkgDate'].apply( lambda x : "01-07" if (x.day <=  7) else "08-14" if (x.day <= 14)
                                                     else "15-21" if (x.day <= 21) else "22-28" if (x.day <= 28)
                                                     else "29-31" )

Cancellation_final['CancelledDate.1'] = pd.to_datetime( Cancellation_final['CancelledDate.1'] , format = '%d-%m-%Y')
Cancellation_final['Week'] = Cancellation_final['CancelledDate.1'].apply( lambda x : "01-07" if (x.day <=  7) else "08-14" if (x.day <= 14)
                                                     else "15-21" if (x.day <= 21) else "22-28" if (x.day <= 28)
                                                     else "29-31" )

In [18]:
Cancellation_final.columns

Index(['FlightNumber', 'DepartureStation', 'ArrivalStation', 'DepartureDate',
       'BookingDate', 'CancelledDate', 'ChannelType', 'FlightStatus', 'PNR',
       'PassengerLastNameAndFirstName', 'BaseFare', 'ServiceChargeAmount',
       'FuelTax', 'OtherTaxes', 'FareClass', 'Cabin', 'FareBasis', 'PromoCode',
       'PromoCodeAmount', 'BookingStatus', 'PassengerStatus', 'PassengerType',
       'BookingAgentID', 'SourceOrganizationCode',
       'SourceOrganizationCodeName', 'PaymentOrganizationCode',
       'OrganizationGroup', 'PaymentOrganizationName', 'AgentRole',
       'TransactionDate', 'DepartureDate.1', 'BookingDate.1',
       'CancelledDate.1', 'Dep_Month', 'Booked_Month', 'Cancellation_Month',
       'Cancelled_Collected_Revenue', 'AgentId', 'ParentId', 'TAName',
       'Region', 'City', 'Sub_Channel', 'Sub_MainChannel',
       'NEWMainChannelforCommercialTeam', 'Charter_Noncharter', 'F1', 'F2',
       'F3', 'F4', 'Week', 'Pax', 'MainChannelBasedOnParentID', 'Sector',
       'N

In [19]:
Booking_final.columns

Index(['FlightNumber', 'DepartureStation', 'ArrivalStation', 'DepartureDate',
       'BookingDate', 'ModifiedDate', 'CancelledDate', 'ChannelType',
       'FlightStatus', 'PNR', 'PassengerLastNameAndFirstName', 'BaseFare',
       'ServiceChargeAmount', 'FuelTax', 'OtherTaxes', 'FareClass', 'Cabin',
       'FareBasis', 'PromoCode', 'PromoCodeAmount', 'PassengerStatus',
       'PassengerType', 'BookingAgentID', 'SourceOrganizationCode',
       'SourceOrganizationCodeName', 'PaymentOrganizationCode',
       'OrganizationGroup', 'PaymentOrganizationName', 'AgentRole', 'Pax',
       'Gross_Rev', 'Net_Rev', 'Gross_Rev_inCrs', 'Sector', 'Sector1',
       'DOM_INT', 'BkgMonth', 'DepMonth', 'BkgDate', 'AgentID', 'ParentID',
       'IDType', 'Region', 'City', 'TAName', 'Sub_Channel', 'Sub_MainChannel',
       'RGM', 'ASM', 'CM', 'BDM', 'PromoCodeType', 'CorporateName', 'BDM_Corp',
       'CM_Corp', 'City_Corp', 'Region_Corp', 'Grp_FIT', 'MainChannel',
       'Country_Name', 'Global_Region', 'Dat

### DOM INT Summary

In [20]:
#divisor values corresponding to the days in each week
divisors = Booking_final.groupby('Week')['BkgDate'].nunique()

In [21]:
divisors

Week
01-07    7
08-14    7
15-21    7
22-28    7
29-31    3
Name: BkgDate, dtype: int64

In [22]:
#Average booking Pax for FIT
fit_avg_pax = Booking_final.loc[Booking_final['Grp_FIT'] == 'FIT'].pivot_table(index='DOM_INT', columns='Week', values='Pax', aggfunc='sum') / divisors
fit_avg_pax

Week,01-07,08-14,15-21,22-28,29-31
DOM_INT,,,,,
DOM,27872.142857,25331.285714,21456.0,18159.000000,21724.000000
INT,4268.571429,3545.285714,2513.0,2120.142857,2582.666667


In [23]:
#Average Booking Revenue for FIT
fit_avg_rev = Booking_final.loc[Booking_final['Grp_FIT'] == 'FIT'].pivot_table(index='DOM_INT', columns='Week', values='Gross_Rev_inCrs', aggfunc='sum') / divisors
fit_avg_rev

Week,01-07,08-14,15-21,22-28,29-31
DOM_INT,,,,,
DOM,13.945144,12.844027,11.486512,9.559383,10.943107
INT,5.141994,4.405025,3.331747,2.985201,3.446707


In [24]:
#Average Booking Pax for GnS
GnS_avg_pax = Booking_final.loc[Booking_final['Grp_FIT'] != 'FIT'].pivot_table(index='DOM_INT', columns='Week', values='Pax', aggfunc='sum') / divisors
GnS_avg_pax

Week,01-07,08-14,15-21,22-28,29-31
DOM_INT,,,,,
DOM,5999.000000,10451.142857,8719.571429,5474.857143,6436.333333
INT,719.428571,1436.285714,1341.571429,657.285714,1391.333333


In [25]:
#Average Booking Revenue for GnS
GnS_avg_rev = Booking_final.loc[Booking_final['Grp_FIT'] != 'FIT'].pivot_table(index='DOM_INT', columns='Week', values='Gross_Rev_inCrs', aggfunc='sum') / divisors
GnS_avg_rev

Week,01-07,08-14,15-21,22-28,29-31
DOM_INT,,,,,
DOM,2.954749,5.467549,4.417777,2.552559,3.733752
INT,0.948256,1.798074,1.648650,0.997778,1.783323


In [26]:
#Average Cancellation pax for FIT
fit_can_pax = Cancellation_final.loc[Cancellation_final['FIT_Grp_Series'] == 'FIT'].pivot_table(columns='Week', values='Pax', aggfunc='sum') / divisors
fit_can_pax

Week,01-07,08-14,15-21,22-28,29-31
Pax,3175.0,3896.0,3591.285714,3765.857143,3152.0


In [27]:
#Average cancellation pax for GnS
Gns_can_pax = Cancellation_final.loc[Cancellation_final['FIT_Grp_Series'] != 'FIT'].pivot_table(columns='Week', values='Pax', aggfunc='sum') / divisors
Gns_can_pax

Week,01-07,08-14,15-21,22-28,29-31
Pax,1347.142857,1615.0,4273.285714,4806.571429,2273.666667


In [28]:
#Exporting the Average values to the Intermediary file.

with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    fit_avg_pax.to_excel(writer1, sheet_name="DOM&INT", startrow=3,startcol=1)

with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    fit_avg_rev.to_excel(writer1, sheet_name="DOM&INT", startrow=8,startcol=1)
    
with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    GnS_avg_pax.to_excel(writer1, sheet_name="DOM&INT", startrow=13,startcol=1)

with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    GnS_avg_rev.to_excel(writer1, sheet_name="DOM&INT", startrow=18,startcol=1)
    
with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    fit_can_pax.to_excel(writer1, sheet_name="DOM&INT", startrow=23,startcol=1)
    
with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    Gns_can_pax.to_excel(writer1, sheet_name="DOM&INT", startrow=27,startcol=1)

### Channel & Region Tab

#### Channel - 

#### Previous Year - 

In [29]:
#prev year pax & revenue
prevyear_CH_paxrev = df_prev_year.groupby('NEWMainChannelforCommercialTeam').agg({'PAX':'sum','GrossRev':lambda x : x.sum() / 10000000})
prevyear_CH_paxrev = prevyear_CH_paxrev.rename(columns={'PAX':'Pax','GrossRev':'Rev'})

prevyear_CH_paxrev_final = pd.merge(CHMast,prevyear_CH_paxrev,how='left',left_on='ChnName',right_on='NEWMainChannelforCommercialTeam')[['ChnName','Pax','Rev']]

#prev year pax
prev_year_chanpax = sqldf("select Pax as Actual from prevyear_CH_paxrev_final")

wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['Region & Channel']
ws['C13'].options(index=False,header=False).value = prev_year_chanpax

#prev year revenue
prev_year_chanrev = sqldf("select Rev as Actual from prevyear_CH_paxrev_final")
ws['R13'].options(index=False,header=False).value = prev_year_chanrev

#### Previous Month

In [30]:
#prev month pax & rev
prevmonth_CH_paxrev = df_prev_mnth.groupby("NEWMainChannelforCommercialTeam").agg({'PAX':'sum','GrossRev':lambda x : x.sum() / 10000000})
prevmonth_CH_paxrev = prevmonth_CH_paxrev.rename(columns={'PAX':'Pax','GrossRev':'Rev'})

prevmonth_CH_paxrev_final = pd.merge(CHMast,prevmonth_CH_paxrev,how='left',left_on = 'ChnName',right_on='NEWMainChannelforCommercialTeam')[['ChnName','Pax','Rev']]

prev_month_chanpax = sqldf("select Pax as Actual from prevmonth_CH_paxrev_final")

wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['Region & Channel']
ws['D13'].options(index=False,header=False).value = prev_month_chanpax

prev_month_chanrev = sqldf("select Rev as Actual from prevmonth_CH_paxrev_final")

ws['S13'].options(index=False,header=False).value = prev_month_chanrev

In [31]:
#current month pax & revenue
Channelpaxrev = Booking_final.pivot_table(
    index='MainChannel',
    columns='MTD/FWD',
    values=['Pax', 'Gross_Rev_inCrs'],
    aggfunc={'Pax': 'sum', 'Gross_Rev_inCrs': 'sum'},
    fill_value=0
).reset_index()

Channelpaxrev.columns = ['MainChannel', 'FWD_Rev', 'MTD_Rev', 'FWD_Pax', 'MTD_Pax']
Channelpaxrev['Gross_Rev_inCrs'] = Channelpaxrev['FWD_Rev'] + Channelpaxrev['MTD_Rev']

Channelpaxrevfinal = pd.merge(CHMast,Channelpaxrev,how='left',left_on='ChnName',right_on='MainChannel')[['ChnName','MTD_Pax','FWD_Pax','Gross_Rev_inCrs']]


channel_pax = sqldf("select MTD_Pax as [Actual MTD], FWD_Pax as [Actual FWD] from Channelpaxrevfinal")

channel_rev = sqldf("select Gross_Rev_inCrs from Channelpaxrevfinal")

In [32]:
Channelpaxrevfinal

,ChnName,MTD_Pax,FWD_Pax,Gross_Rev_inCrs
0,PORTALS B2C,211193,148519,190.640559
1,PORTALS B2B,109998,77099,113.484988
2,RETAIL,25565,264916,178.791427
3,CORPORATE,110444,88795,138.962325
4,GLOBAL OTA,1929,2206,3.406311
5,WEB,13748,13680,23.411752
6,DIRECT,7376,1387,2.414281


In [33]:
wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['Region & Channel']
ws['H13'].options(index=False,header=False).value = channel_pax

ws['U13'].options(index=False,header=False).value = channel_rev

In [34]:
#Channel wise Can Pax
Can_channelpax = Cancellation_final.groupby("NEWMainChannelforCommercialTeam").agg({'Pax':'sum'})

Can_channelpaxfinal = pd.merge(CHMast,Can_channelpax,how='left',left_on='ChnName',right_on='NEWMainChannelforCommercialTeam')[['Pax']]

wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['Region & Channel']
ws['AI13'].options(index=False,header=False).value = Can_channelpaxfinal

#### Region - 

In [35]:
#prev year pax & rev
prevyear_RN_paxrev = df_prev_year.groupby("Global_Region").agg({'PAX':'sum','GrossRev':lambda x : x.sum()/10000000})
prevyear_RN_paxrev = prevyear_RN_paxrev.rename(columns = {'PAX':'Pax','GrossRev':'Rev'})

prevyear_RN_paxrev_final = pd.merge(RNMast,prevyear_RN_paxrev,how='left',left_on='RegName',right_on='Global_Region')[['RegName','Pax','Rev']]

prev_year_regionpax = sqldf("select Pax as Actual from prevyear_RN_paxrev_final")

wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['Region & Channel']
ws['C27'].options(index=False,header=False).value = prev_year_regionpax

prev_year_regrev = sqldf("select Rev as Actual from prevyear_RN_paxrev_final")

ws['R27'].options(index=False,header=False).value = prev_year_regrev

In [36]:
#prev month pax & rev
prevmonth_RN_paxrev = df_prev_mnth.groupby("Global_Region").agg({'PAX':'sum','GrossRev':lambda x:x.sum()/10000000})
prevmonth_RN_paxrev = prevmonth_RN_paxrev.rename(columns={'PAX':'Pax','GrossRev':'Rev'})

prevmonth_RN_paxrev_final = pd.merge(RNMast,prevmonth_RN_paxrev,how='left',left_on='RegName',right_on='Global_Region')[['RegName','Pax','Rev']]

prev_month_regpax = sqldf("select Pax as Actual from prevmonth_RN_paxrev_final")

wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['Region & Channel']
ws['D27'].options(index=False,header=False).value = prev_month_regpax

prev_month_regrev = sqldf("select Rev as Actual from prevmonth_RN_paxrev_final")

ws['S27'].options(index=False,header=False).value = prev_month_regrev

In [37]:
#current month pax & rev
Regionpaxrev = Booking_final.pivot_table(
    index='Region',
    columns='MTD/FWD',
    values=['Pax', 'Gross_Rev_inCrs'],
    aggfunc={'Pax': 'sum', 'Gross_Rev_inCrs': 'sum'},
    fill_value=0
).reset_index()

Regionpaxrev.columns = ['Region', 'FWD_Rev', 'MTD_Rev', 'FWD_Pax', 'MTD_Pax']
Regionpaxrev['Gross_Rev_inCrs'] = Regionpaxrev['FWD_Rev'] + Regionpaxrev['MTD_Rev']


regionpaxrevfinal = pd.merge(RNMast,Regionpaxrev,how='left',left_on='RegName',right_on='Region')[['RegName','MTD_Pax','FWD_Pax','Gross_Rev_inCrs']]

region_pax = sqldf("select MTD_Pax as [Actual MTD], FWD_Pax as [Actual FWD] from regionpaxrevfinal")

wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['Region & Channel']
ws['H27'].options(index=False,header=False).value = region_pax

region_rev = sqldf("select Gross_Rev_inCrs from regionpaxrevfinal")

ws['U27'].options(index=False,header=False).value = region_rev

In [38]:
regionpaxrevfinal

,RegName,MTD_Pax,FWD_Pax,Gross_Rev_inCrs
0,NORTH,269347,276208,301.268613
1,WEST,125059,137256,165.295560
2,SOUTH,27250,51636,52.058918
3,EAST,22072,98822,69.004394
4,GSA,15406,17613,37.661207
5,WEB,13748,13680,23.411752
6,DIRECT,7371,1387,2.411198


In [39]:
#prev year Group&Series pax & rev
prevyear_RN_GSpaxrev = df_prev_year[df_prev_year['FIT_Grp'] != 'FIT'].groupby("Global_Region").agg({'PAX':'sum','GrossRev':lambda x : x.sum()/10000000})
prevyear_RN_GSpaxrev = prevyear_RN_GSpaxrev.rename(columns = {'PAX':'Pax','GrossRev':'Rev'})

prevyear_RN_GSpaxrev_final = pd.merge(RNMast,prevyear_RN_GSpaxrev,how='left',left_on='RegName',right_on='Global_Region')[['RegName','Pax','Rev']]

prev_year_GSregionpax = sqldf("select Pax as Actual from prevyear_RN_GSpaxrev_final")

wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['Region & Channel']
ws['AH27'].options(index=False,header=False).value = prev_year_GSregionpax

prev_year_GSregrev = sqldf("select Rev as Actual from prevyear_RN_GSpaxrev_final")

ws['AN27'].options(index=False,header=False).value = prev_year_GSregrev

In [40]:
#prev month Group&Series pax & rev
prevmonth_RN_GSpaxrev = df_prev_mnth[df_prev_mnth['FIT_Grp'] != 'FIT'].groupby("Global_Region").agg({'PAX':'sum','GrossRev':lambda x:x.sum()/10000000})
prevmonth_RN_GSpaxrev = prevmonth_RN_GSpaxrev.rename(columns={'PAX':'Pax','GrossRev':'Rev'})

prevmonth_RN_GSpaxrev_final = pd.merge(RNMast,prevmonth_RN_GSpaxrev,how='left',left_on='RegName',right_on='Global_Region')[['RegName','Pax','Rev']]

prev_month_GSregpax = sqldf("select Pax as Actual from prevmonth_RN_GSpaxrev_final")

wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['Region & Channel']
ws['AI27'].options(index=False,header=False).value = prev_month_GSregpax

prev_month_GSregrev = sqldf("select Rev as Actual from prevmonth_RN_GSpaxrev_final")

ws['AO27'].options(index=False,header=False).value = prev_month_GSregrev

In [41]:
#Group & Series Pax & Revenue
RegionGSpaxrevdata = Booking_final[Booking_final['Grp_FIT'] == 'Grp & Series']
                          
RegionGSpaxrevdatafinal = RegionGSpaxrevdata.pivot_table(values = ["Pax","Gross_Rev_inCrs"],index="Region",aggfunc="sum").fillna(0)

#Ordering the data
RegionGSpaxrevdatafinal1 = pd.merge(RNMast,RegionGSpaxrevdatafinal,how='left',left_on='RegName',right_on='Region')
RegionGSpaxrevdatafinal1 = RegionGSpaxrevdatafinal1.rename(columns={'Pax':'GrpPax','Gross_Rev_inCrs':'GrpRev'})

RegionGSpaxrevdatafinal1['GrpPax'] = RegionGSpaxrevdatafinal1['GrpPax'].fillna(0)
RegionGSpaxrevdatafinal1['GrpPax'] = RegionGSpaxrevdatafinal1['GrpPax'].astype(int)

wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['Region & Channel']

GS_region_pax = sqldf("select GrpPax from RegionGSpaxrevdatafinal1")
ws['AJ27'].options(index=False,header=False).value = GS_region_pax

GS_region_rev = sqldf("select GrpRev from RegionGSpaxrevdatafinal1")

ws['AP27'].options(index=False,header=False).value = GS_region_rev

### Sector

In [42]:
#prev year sector pax & revenue
prev_year_sector = df_prev_year.groupby('Sector').agg({'PAX':'sum','GrossRev':lambda x : x.sum() / 10000000})

prev_year_sector_ordered = pd.merge(SectorMast,prev_year_sector,how='left',left_on='Sector',right_on='Sector')[['Sector','PAX','GrossRev']]


prev_year_sector_ordered['PAX'] = prev_year_sector_ordered['PAX'].fillna(0)
prev_year_sector_ordered['GrossRev'] = prev_year_sector_ordered['GrossRev'].fillna(0)

wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['Sector']

prev_yr_sec_pax = sqldf("select PAX from prev_year_sector_ordered")
ws['F13'].options(index=False,header=False).value = prev_yr_sec_pax

prev_yr_sec_rev = sqldf("select GrossRev from prev_year_sector_ordered")

ws['Q13'].options(index=False,header=False).value = prev_yr_sec_rev


In [43]:
#prev month pax & revenue
prev_month_sector = df_prev_mnth.groupby('Sector').agg({'PAX':'sum','GrossRev':lambda x : x.sum() / 10000000})

prev_month_sector_ordered = pd.merge(SectorMast,prev_month_sector,how='left',left_on='Sector',right_on='Sector')[['Sector','PAX','GrossRev']]


prev_month_sector_ordered['PAX'] = prev_month_sector_ordered['PAX'].fillna(0)
prev_month_sector_ordered['GrossRev'] = prev_month_sector_ordered['GrossRev'].fillna(0)

wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['Sector']

prev_mnth_sec_pax = sqldf("select PAX from prev_month_sector_ordered")
ws['G13'].options(index=False,header=False).value = prev_mnth_sec_pax

prev_mnth_sec_rev = sqldf("select GrossRev from prev_month_sector_ordered")

ws['R13'].options(index=False,header=False).value = prev_mnth_sec_rev


In [44]:
#curr month sector pax & revenue
curr_month_sector_unordered = Booking_final.groupby('Sector').agg({'Pax':'sum','Gross_Rev_inCrs': 'sum'})

with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    curr_month_sector_unordered.to_excel(writer1, sheet_name="SectorUnordered", startrow=1,startcol=1)

curr_month_sector_ordered = pd.merge(SectorMast,curr_month_sector_unordered,how='left',left_on='Sector',right_on='Sector')[['Sector','Pax','Gross_Rev_inCrs']]

curr_month_sector_ordered['Pax'] = curr_month_sector_ordered['Pax'].fillna(0)
curr_month_sector_ordered['Gross_Rev_inCrs'] = curr_month_sector_ordered['Gross_Rev_inCrs'].fillna(0)

wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['Sector']

cur_mnth_sec_pax = sqldf("select Pax from curr_month_sector_ordered")
ws['J13'].options(index=False,header=False).value = cur_mnth_sec_pax

cur_mnth_sec_rev = sqldf("select Gross_Rev_inCrs from curr_month_sector_ordered")

ws['U13'].options(index=False,header=False).value = cur_mnth_sec_rev

### Channel Summary

In [45]:
#Channelwise Pax Summary - 
ChannelPax = Booking_final.pivot_table(index='BookingDate',
                                       values='Pax',
                                       columns='MainChannel',
                                       aggfunc='sum',
                                       fill_value=0,
                                       margins=False)

# Reorder the columns
desired_order = ['PORTALS B2C', 'PORTALS B2B', 'RETAIL', 'CORPORATE', 'GLOBAL OTA', 'WEB', 'DIRECT']
ChannelPax = ChannelPax[desired_order]


wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['Day & Weekly - Channel']

#change the cell reference every start of the month
ws['D707'].options(index=False,header=False).value = ChannelPax

In [46]:
#Channelwise Revenue summary
ChannelRev = Booking_final.pivot_table(index='BookingDate',
                                       values='Gross_Rev_inCrs',
                                       columns='MainChannel',
                                       aggfunc='sum',
                                       fill_value=0,
                                       margins=False)

# Reorder the columns
desired_order = ['PORTALS B2C', 'PORTALS B2B', 'RETAIL', 'CORPORATE', 'GLOBAL OTA', 'WEB', 'DIRECT']
ChannelRev = ChannelRev[desired_order]


wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['Day & Weekly - Channel']

#change the cell reference every start of the month
ws['L707'].options(index=False,header=False).value = ChannelRev

### Region Summary

In [47]:
#Regionwise Pax Summary
RegionPax = Booking_final.pivot_table(index='BookingDate',
                                       values='Pax',
                                       columns='Region',
                                       aggfunc='sum',
                                       fill_value=0,
                                       margins=False)

# Reorder the columns
desired_order = ['NORTH', 'WEST', 'SOUTH', 'EAST', 'GSA', 'WEB', 'DIRECT']
RegionPax = RegionPax[desired_order]


wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['Day & Weekly - Region']

#change the cell reference every start of the month
ws['D707'].options(index=False,header=False).value = RegionPax

In [48]:
#Region wise Revenue Summary
Regionrev = Booking_final.pivot_table(index='BookingDate',
                                       values='Gross_Rev_inCrs',
                                       columns='Region',
                                       aggfunc='sum',
                                       fill_value=0,
                                       margins=False)

# Reorder the columns
desired_order = ['NORTH', 'WEST', 'SOUTH', 'EAST', 'GSA', 'WEB', 'DIRECT']
Regionrev = Regionrev[desired_order]


wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['Day & Weekly - Region']

#change the cell reference every start of the month
ws['L707'].options(index=False,header=False).value = Regionrev

### Fit GSS G&S Trend

In [49]:
#FIT pax & rev
# channelwise_fit = sqldf("""select MainChannel, sum(Pax) as FitPax, sum(Gross_Rev_inCrs) as FitRev,
#           sum(case when DOM_INT = "DOM" then Pax else 0 end) as FITDomPax,
#           sum(case when DOM_INT = "DOM" then Gross_Rev_inCrs else 0 end) as FITDomRev,
#           sum(case when DOM_INT = "INT" then Pax else 0 end) as FITIntPax,
#           sum(case when DOM_INT = "INT" then Gross_Rev_inCrs else 0 end) as FITIntRev
#           from Booking_final where Grp_FIT='FIT' and IDType!='GS' and charter_noncharter = 'Non-Charter' 
#           group by MainChannel""")
          
# channelwise_fit_order = sqldf("""select a.ChnName as Channel, b.FitPax as [FIT DOM+INT Pax], b.FitRev as [FIT DOM+INT Rev],
#                                   b.FITDomPax as [FIT DOM Pax], b.FITDomRev as [FIT DOM Rev], b.FITIntPax as [FIT INT Pax], b.FITIntRev as [FIT INT Rev]
#                                   from CHMast a left join channelwise_fit b on a.ChnName = b.MainChannel""")

# wb = xw.Book( report_file_path )
# wks = wb.sheets
# ws = wks['FIT_GSS_G&S-Trend']
# ws['B40'].options(index=False).value = channelwise_fit_order

In [50]:
#FIT pax & rev
channelwise_fit = Booking_final[(Booking_final['Grp_FIT'] == 'FIT') & (Booking_final['IDType'] != 'GS')] \
    .pivot_table(index='MainChannel',
                 values=['Pax', 'Gross_Rev_inCrs'],
                 columns='DOM_INT',
                 aggfunc={'Pax': 'sum', 'Gross_Rev_inCrs': 'sum'},
                 fill_value=0,
                 )

channelwise_fit.columns = ['_'.join(str(s).strip() for s in col if s) for col in channelwise_fit.columns]

channelwise_fit_order = pd.merge(CHMast,channelwise_fit,how='left',left_on='ChnName',right_on='MainChannel')[['ChnName','Gross_Rev_inCrs_DOM',
                                                                                                             'Gross_Rev_inCrs_INT','Pax_DOM','Pax_INT']]

channelwise_fit_order.rename(columns={'Gross_Rev_inCrs_DOM':'FIT_DOM_Rev','Gross_Rev_inCrs_INT':'FIT_INT_Rev',
                                      'Pax_DOM':'FIT_DOM_Pax','Pax_INT':'FIT_INT_Pax'},inplace=True)


wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['FIT_GSS_G&S-Trend']
ws['B40'].options(index=False).value = channelwise_fit_order

In [51]:
# # GSS Pax and Revenue
# channelwise_gss = sqldf("""select MainChannel, sum(Pax) as GSSPax, sum(Gross_Rev_inCrs) as GSSRev,
#           sum(case when DOM_INT = "DOM" then Pax else 0 end) as GSSDomPax,
#           sum(case when DOM_INT = "DOM" then Gross_Rev_inCrs else 0 end) as GSSDomRev,
#           sum(case when DOM_INT = "INT" then Pax else 0 end) as GSSIntPax,
#           sum(case when DOM_INT = "INT" then Gross_Rev_inCrs else 0 end) as GSSIntRev
#           from Booking_final where Grp_FIT='FIT' and IDType='GS' and charter_noncharter = 'Non-Charter' 
#           group by MainChannel""")
          
# channelwise_gss_order = sqldf("""select a.ChnName as Channel, b.GSSPax as [GSS DOM+INT Pax], b.GSSRev as [GSS DOM+INT Rev],
#                                   b.GSSDomPax as [GSS DOM Pax], b.GSSDomRev as [GSS DOM Rev], b.GSSIntPax as [GSS INT Pax], b.GSSIntRev as [GSS INT Rev]
#                                   from CHMast a left join channelwise_gss b on a.ChnName = b.MainChannel""")

# wb = xw.Book( report_file_path )
# wks = wb.sheets
# ws = wks['FIT_GSS_G&S-Trend']
# ws['K40'].options(index=False).value = channelwise_gss_order

In [52]:
# GSS Pax and Revenue
channelwise_gss = Booking_final[(Booking_final['Grp_FIT'] == 'FIT') & (Booking_final['IDType'] == 'GS')] \
    .pivot_table(index='MainChannel',
                 values=['Pax', 'Gross_Rev_inCrs'],
                 columns='DOM_INT',
                 aggfunc={'Pax': 'sum', 'Gross_Rev_inCrs': 'sum'},
                 fill_value=0,
                 )

channelwise_gss.columns = ['_'.join(str(s).strip() for s in col if s) for col in channelwise_gss.columns]

channelwise_gss_order = pd.merge(CHMast,channelwise_gss,how='left',left_on='ChnName',right_on='MainChannel')[['ChnName','Gross_Rev_inCrs_DOM',
                                                                                                             'Gross_Rev_inCrs_INT','Pax_DOM','Pax_INT']]

channelwise_gss_order.rename(columns={'Gross_Rev_inCrs_DOM':'GSS_DOM_Rev','Gross_Rev_inCrs_INT':'GSS_INT_Rev',
                                      'Pax_DOM':'GSS_DOM_Pax','Pax_INT':'GSS_INT_Pax'},inplace=True)

wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['FIT_GSS_G&S-Trend']
ws['I40'].options(index=False).value = channelwise_gss_order

In [53]:
# GnS Pax and Revenue
# channelwise_gns = sqldf("""select MainChannel, sum(Pax) as GnSPax, sum(Gross_Rev_inCrs) as GnSRev,
#           sum(case when DOM_INT = "DOM" then Pax else 0 end) as GnSDomPax,
#           sum(case when DOM_INT = "DOM" then Gross_Rev_inCrs else 0 end) as GnSDomRev,
#           sum(case when DOM_INT = "INT" then Pax else 0 end) as GnSIntPax,
#           sum(case when DOM_INT = "INT" then Gross_Rev_inCrs else 0 end) as GnSIntRev
#           from Booking_final where Grp_FIT='Grp & Series' and charter_noncharter = 'Non-Charter' 
#           group by MainChannel""")
          
# channelwise_gns_order = sqldf("""select a.ChnName as Channel, b.GnSPax as [GnS DOM+INT Pax], b.GnSRev as [GnS DOM+INT Rev],
#                                   b.GnSDomPax as [GnS DOM Pax], b.GnSDomRev as [GnS DOM Rev], b.GnSIntPax as [GnS INT Pax], b.GnSIntRev as [GnS INT Rev]
#                                   from CHMast a left join channelwise_gns b on a.ChnName = b.MainChannel""")

# wb = xw.Book( report_file_path )
# wks = wb.sheets
# ws = wks['FIT_GSS_G&S-Trend']
# ws['T40'].options(index=False).value = channelwise_gns_order

In [54]:
# GnS Pax and Revenue
channelwise_gns = Booking_final[Booking_final['Grp_FIT'] == 'Grp & Series'] \
    .pivot_table(index='MainChannel',
                 values=['Pax', 'Gross_Rev_inCrs'],
                 columns='DOM_INT',
                 aggfunc={'Pax': 'sum', 'Gross_Rev_inCrs': 'sum'},
                 fill_value=0,
                 )

channelwise_gns.columns = ['_'.join(str(s).strip() for s in col if s) for col in channelwise_gns.columns]

channelwise_gns_order = pd.merge(CHMast,channelwise_gns,how='left',left_on='ChnName',right_on='MainChannel')[['ChnName','Gross_Rev_inCrs_DOM',
                                                                                                             'Gross_Rev_inCrs_INT','Pax_DOM','Pax_INT']]

channelwise_gns_order.rename(columns={'Gross_Rev_inCrs_DOM':'GnS_DOM_Rev','Gross_Rev_inCrs_INT':'GnS_INT_Rev',
                                      'Pax_DOM':'GnS_DOM_Pax','Pax_INT':'GnS_INT_Pax'},inplace=True)


wb = xw.Book( report_file_path )
wks = wb.sheets
ws = wks['FIT_GSS_G&S-Trend']
ws['P40'].options(index=False).value = channelwise_gns_order

### RCC

In [55]:
df_prev_year.columns

Index(['BookDate', 'DOM/INT', 'Global_Region_GSA', 'Global_Region', 'City',
       'NEWMainChannelforCommercialTeam', 'Sector', 'Country', 'FIT_Grp',
       'PAX', 'GrossRev'],
      dtype='object')

#### Prev Year

In [56]:
#region, city and channel pax and revenue 
regioncitychannelPrevYear = sqldf("""select Global_Region_GSA, City, NEWMainChannelforCommercialTeam, Global_Region, 
                          sum(Pax) as Pax, sum(GrossRev)/10000000 as GrossRevenue from df_prev_year  group by Global_Region_GSA, City, 
                       NEWMainChannelforCommercialTeam""")

with pd.ExcelWriter(r"PrevYear.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer:
    regioncitychannelPrevYear.to_excel(writer, sheet_name="RCCPaxRev", startrow=0,startcol=0, index=False)


regioncitychannelPrevYearWD = sqldf(""" Select Global_Region,sum(Pax) as PAX, sum(GrossRevenue) as GrossRevenue  from regioncitychannelPrevYear where
                                       Global_Region In ('WEB','DIRECT') group by
                                       Global_Region  """)
                                       

with pd.ExcelWriter(r"PrevYear.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer:
    regioncitychannelPrevYearWD.to_excel(writer, sheet_name="RCCPax", startrow=1,startcol=10, index=False)

In [57]:
#region, city and channel pax and revenue merging with master
df6 = pd.read_excel(r"PrevYear.xlsx", sheet_name= 'RCCPaxRev' )

#region, city and channel pax and revenue key calculation
df6['key1'] = df6['Global_Region_GSA']+df6['City']+df6['NEWMainChannelforCommercialTeam']

In [58]:
res1 = sqldf(""" 
              Select m.Region,m.City,m.Channel, df6.Pax, df6.GrossRevenue, df6.key1
              From RegionCityChannelcombo m
              Left Outer Join df6 on 
              (m.Region = df6.Global_Region_GSA AND m.City = df6.City AND m.Channel = df6.NEWMainChannelforCommercialTeam)
              Order BY
              m.RegionCode ASC, m.CityCode ASC, m.ChannelCode ASC
              """)

with pd.ExcelWriter(r"PrevYear.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer:
    res1.to_excel(writer, sheet_name="OrderedRCC", startrow=0,startcol=0, index=False)

In [59]:
df8 = pd.read_excel(r"PrevYear.xlsx", sheet_name= 'OrderedRCC')


res4 = sqldf("""select a.*, b.Pax, b.GrossRevenue
             from RegionCityChannelkey a left join  df8 b on a.Key=b.key1 """)

with pd.ExcelWriter(r"PrevYear.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer:
    res4.to_excel(writer, sheet_name="RCCPax", startrow=0,startcol=1, index=False)

### Prev Month

In [60]:
#region, city and channel pax and revenue 
regioncitychannlepaxPrevmon = sqldf("""select Region, City, NEWMainChannelforCommercialTeam, 
                      Global_Region,  sum(PAX) as Pax, sum(GrossRev)/(10000000) as GrossRevenue from df_prev_mnth  
                      group by Region, City, NEWMainChannelforCommercialTeam""")


with pd.ExcelWriter("PrevMonthintermediaryfile.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer:
    regioncitychannlepaxPrevmon.to_excel(writer, sheet_name="RCCPaxRev", startrow=0,startcol=0, index=False)


regioncitychannlepaxPrevmonWD = sqldf(""" Select Region,sum(Pax) as PAX, sum(GrossRevenue) as GrossRevenue  from regioncitychannlepaxPrevmon where
                                       Region In ('WEB','DIRECT') group by
                                       Region  """)
                                       
with pd.ExcelWriter("PrevMonthintermediaryfile.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer:
    regioncitychannlepaxPrevmonWD.to_excel(writer, sheet_name="RCCPax", startrow=1,startcol=13, index=False)

In [61]:
#region, city and channel pax and revenue merging with master
df6 = pd.read_excel("PrevMonthintermediaryfile.xlsx", sheet_name= 'RCCPaxRev' )

#region, city and channel pax and revenue key calculation
df6['key1'] = df6['Region']+df6['City']+df6['NEWMainChannelforCommercialTeam']

In [62]:
res1 = sqldf(""" 
              Select m.Region,m.City,m.Channel, df6.Pax, df6.GrossRevenue, df6.key1
              From RegionCityChannelcombo m
              Left Outer Join df6 on 
              (m.Region = df6.Region AND m.City = df6.City AND m.Channel = df6.NEWMainChannelforCommercialTeam)
              Order BY
              m.RegionCode ASC, m.CityCode ASC, m.ChannelCode ASC
              """)
with pd.ExcelWriter("PrevMonthintermediaryfile.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer:
    res1.to_excel(writer, sheet_name="OrderedRCC", startrow=0,startcol=0, index=False)

In [63]:
df8 = pd.read_excel("PrevMonthintermediaryfile.xlsx", sheet_name= 'OrderedRCC')


res4 = sqldf("""select a.*, b.Pax, b.GrossRevenue
             from RegionCityChannelkey a left join  df8 b on a.Key=b.key1 """)

with pd.ExcelWriter("PrevMonthintermediaryfile.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer:
    res4.to_excel(writer, sheet_name="RCCPax", startrow=0,startcol=0, index=False)

#### Current Month

In [64]:
# data for rcc tab
RCCdata = Booking_final.copy()
RCCdata['Region_RCC'] = RCCdata.apply(lambda x: x['Global_Region'] if x['Region'] == 'GSA' else x['Region'], axis=1)
RCCdata['Region_RCC'] = RCCdata['Region_RCC'].str.upper()

Rcc = sqldf("""select Region_RCC, City, MainChannel,Global_Region,
                sum(case when [MTD/FWD]= 'MTD' then Pax else 0 end) as MTD_Pax,
                sum(case when [MTD/FWD]= 'FWD' then Pax else 0 end) as FWD_Pax,
                sum(Gross_Rev_inCrs) as Rev from RCCdata  
                group by Region_RCC,City,MainChannel
                               """) 

In [65]:
Rcc['RCCkey']=Rcc['Region_RCC']+Rcc['City']+Rcc["MainChannel"]

In [66]:
RCC_booking = sqldf("""select Region_RCC,City,MainChannel,
                    MTD_Pax,FWD_Pax,Rev,RCCkey from Rcc
                    """)

In [67]:
RCC_Final= sqldf("""select df3.*, b.Region_RCC as Region,b.City as city,b.MainChannel as Channel,
                    b.MTD_Pax as MTD_Pax, b.FWD_Pax as FWD_Pax,b.Rev as Rev
             from RegionCityChannelkey df3 left join RCC_booking b on df3.Key=b.RCCkey
             """)
RCC_Final['MTD_Pax'] = RCC_Final['MTD_Pax'].fillna(0)
RCC_Final['FWD_Pax'] = RCC_Final['FWD_Pax'].fillna(0)

RCC_Final['MTD_Pax'] = RCC_Final['MTD_Pax'].astype(int)
RCC_Final['FWD_Pax'] = RCC_Final['FWD_Pax'].astype(int)



with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    RCC_Final.to_excel(writer1, sheet_name="RCC", startrow=1,startcol=1, index=False)

### B2C

In [68]:
#Filtering Mainchannel by B2C 
df_B2C = Booking_final[Booking_final['MainChannel'] == 'PORTALS B2C']

In [69]:
df_B2C['TA_Name'] = df_B2C.apply(lambda x : 'OTHERS' if x['TAName'] not in B2CMaster['TAName'].values else x['TAName'],axis = 1 )

In [70]:
#B2C wise pax summary
df_B2C_Pax = df_B2C.pivot_table(index = ['TA_Name'] , columns = ['BookingDate'] , values = ['Pax'],aggfunc = np.sum)
df_B2C_Pax.columns = ['_'.join(str(s).strip() for s in col if s) for col in df_B2C_Pax.columns]

In [71]:
#Ordering the data
df_B2C_Pax_ordered = pd.merge(B2CMaster,df_B2C_Pax,how='left',left_on='TAName',right_on='TA_Name')

with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    df_B2C_Pax_ordered.to_excel(writer1, sheet_name="B2C", startrow=1,startcol=1, index=False)

In [72]:
#B2C wise rev summary
df_B2C_Rev = df_B2C.pivot_table(index = ['TA_Name'] , columns = ['BookingDate'] , values = ['Gross_Rev_inCrs'],aggfunc = np.sum)
df_B2C_Rev.columns = ['_'.join(str(s).strip() for s in col if s) for col in df_B2C_Rev.columns]

In [73]:
#ordering the data
df_B2C_Rev_ordered = pd.merge(B2CMaster,df_B2C_Rev,how='left',left_on='TAName',right_on='TA_Name')

with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    df_B2C_Rev_ordered.to_excel(writer1, sheet_name="B2C", startrow=23,startcol=1, index=False)

#### B2C Average weekly pax

In [74]:
divisors = Booking_final.groupby('Week')['BkgDate'].nunique()

In [75]:
#B2C week wise average pax
B2C_avg_pax = df_B2C.pivot_table(index='TA_Name', columns='Week', values='Pax', aggfunc='sum') / divisors

B2C_avg_pax_ordered = pd.merge(B2CMaster,B2C_avg_pax,how='left',left_on='TAName',right_on='TA_Name')

with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    B2C_avg_pax_ordered.to_excel(writer1, sheet_name="B2C_Average", startrow=1,startcol=1, index=False)

In [76]:
#B2C wise average revenue 
B2C_avg_rev = df_B2C.pivot_table(index='TA_Name', columns='Week', values='Gross_Rev_inCrs', aggfunc='sum') / divisors

B2C_avg_rev_ordered = pd.merge(B2CMaster,B2C_avg_rev,how='left',left_on='TAName',right_on='TA_Name')

with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    B2C_avg_rev_ordered.to_excel(writer1, sheet_name="B2C_Average", startrow=22,startcol=1, index=False)

#### B2C Can Pax

In [77]:
#Fltering can data by B2C
df_B2C_can = Cancellation_final[Cancellation_final['NEWMainChannelforCommercialTeam'] == 'PORTALS B2C']

In [78]:
df_B2C_can['TA_Name'] = df_B2C_can.apply(lambda x : 'OTHERS' if x['TAName'] not in B2CMaster['TAName'].values else x['TAName'],axis = 1 )

In [79]:
#B2C can pax summary
df_B2C_can_Pax = df_B2C_can.pivot_table(index = ['TA_Name'] , columns = ['CancelledDate.1'] , values = ['Pax'],aggfunc = np.sum)
df_B2C_can_Pax.columns = ['_'.join(str(s).strip() for s in col if s) for col in df_B2C_can_Pax.columns]

In [80]:
#Ordering the data
df_B2C_can_Pax_ordered = pd.merge(B2CMaster,df_B2C_can_Pax,how='left',left_on='TAName',right_on='TA_Name')

with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    df_B2C_can_Pax_ordered.to_excel(writer1, sheet_name="B2C_Can", startrow=1,startcol=1, index=False)

#### Avg B2C Can Pax 

In [81]:
#B2C wise average can pax
B2C_avg_can_pax = df_B2C_can.pivot_table(index='TA_Name', columns='Week', values='Pax', aggfunc='sum') / divisors

B2C_avg_can_pax_ordered = pd.merge(B2CMaster,B2C_avg_can_pax,how='left',left_on='TAName',right_on='TA_Name')

with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    B2C_avg_can_pax_ordered.to_excel(writer1, sheet_name="B2C_B2B_Average_Can", startrow=1,startcol=1, index=False)

### B2B

In [82]:
#Filtering Mainchannel by B2B
df_B2B = Booking_final[Booking_final['MainChannel'] == 'PORTALS B2B']

In [83]:
df_B2B['TA_Name'] = df_B2B.apply(lambda x : 'OTHERS' if x['TAName'] not in B2BMaster['TAName'].values else x['TAName'],axis = 1 )

In [84]:
#B2B pax summary
df_B2B_Pax = df_B2B.pivot_table(index = ['TA_Name'] , columns = ['BookingDate'] , values = ['Pax'],aggfunc = np.sum)
df_B2B_Pax.columns = ['_'.join(str(s).strip() for s in col if s) for col in df_B2B_Pax.columns]

In [85]:
#ordering data
df_B2B_Pax_ordered = pd.merge(B2BMaster,df_B2B_Pax,how='left',left_on='TAName',right_on='TA_Name')

with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    df_B2B_Pax_ordered.to_excel(writer1, sheet_name="B2B", startrow=1,startcol=1, index=False)

In [86]:
#B2B revenue summary
df_B2B_Rev = df_B2B.pivot_table(index = ['TA_Name'] , columns = ['BookingDate'] , values = ['Gross_Rev_inCrs'],aggfunc = np.sum)
df_B2B_Rev.columns = ['_'.join(str(s).strip() for s in col if s) for col in df_B2B_Rev.columns]

In [87]:
#Ordering the data
df_B2B_Rev_ordered = pd.merge(B2BMaster,df_B2B_Rev,how='left',left_on='TAName',right_on='TA_Name')

with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    df_B2B_Rev_ordered.to_excel(writer1, sheet_name="B2B", startrow=26,startcol=1, index=False)

#### B2B Weekly Average Pax/Rev

In [88]:
#B2B week wise average pax summary
B2B_avg_pax = df_B2B.pivot_table(index='TA_Name', columns='Week', values='Pax', aggfunc='sum') / divisors

B2B_avg_pax_ordered = pd.merge(B2BMaster,B2B_avg_pax,how='left',left_on='TAName',right_on='TA_Name')

with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    B2B_avg_pax_ordered.to_excel(writer1, sheet_name="B2B_Average", startrow=1,startcol=1, index=False)

In [89]:
#B2B week wise average rev summary
B2B_avg_rev = df_B2B.pivot_table(index='TA_Name', columns='Week', values='Gross_Rev_inCrs', aggfunc='sum') / divisors

B2B_avg_rev_ordered = pd.merge(B2BMaster,B2B_avg_rev,how='left',left_on='TAName',right_on='TA_Name')

with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    B2B_avg_rev_ordered.to_excel(writer1, sheet_name="B2B_Average", startrow=26,startcol=1, index=False)

#### B2B Can Pax

In [90]:
#Filtering can data for B2B
df_B2B_Can = Cancellation_final[Cancellation_final['NEWMainChannelforCommercialTeam'] == 'PORTALS B2B']

In [91]:
df_B2B_Can['TA_Name'] = df_B2B_Can.apply(lambda x : 'OTHERS' if x['TAName'] not in B2BMaster['TAName'].values else x['TAName'],axis = 1 )

In [92]:
#B2B can pax summary
df_B2B_Can_Pax = df_B2B_Can.pivot_table(index = ['TA_Name'] , columns = ['CancelledDate.1'] , values = ['Pax'],aggfunc = np.sum)
df_B2B_Can_Pax.columns = ['_'.join(str(s).strip() for s in col if s) for col in df_B2B_Can_Pax.columns]

In [93]:
#Ordering the data
df_B2B_Can_Pax_ordered = pd.merge(B2BMaster,df_B2B_Can_Pax,how='left',left_on='TAName',right_on='TA_Name')

with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    df_B2B_Can_Pax_ordered.to_excel(writer1, sheet_name="B2B_Can", startrow=1,startcol=1, index=False)

#### Weekly Avg B2B Can Pax

In [94]:
#B2B wise average can pax summary
B2B_avg_can_pax = df_B2B_Can.pivot_table(index='TA_Name', columns='Week', values='Pax', aggfunc='sum') / divisors

B2B_avg_can_pax_ordered = pd.merge(B2BMaster,B2B_avg_can_pax,how='left',left_on='TAName',right_on='TA_Name')

with pd.ExcelWriter("IntermediaryfiledailyMIS.xlsx",mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer1:
    B2B_avg_can_pax_ordered.to_excel(writer1, sheet_name="B2C_B2B_Average_Can", startrow=22,startcol=1, index=False)

In [95]:
end_time = datetime.now()

In [96]:
print('Total Time to Excecute the code is {}'.format(end_time-start_time))

Total Time to Excecute the code is 0:02:39.894165
